In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
import os
# initialize the models

models = ['gpt-3.5-turbo-16k', 'gpt-4','gpt-4-1106-preview',"text-davinci-003"]
model = models[1]

openai = OpenAI(
    model_name=model,
    # openai_api_key= os.environ["OPENAI_API_KEY"]
    temperature=0.2
)

/Users/ianderrington/miniconda3/envs/genai/lib/python3.10/site-packages/langchain_community/llms/openai.py:248: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/Users/ianderrington/miniconda3/envs/genai/lib/python3.10/site-packages/langchain_community/llms/openai.py:1057: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
BASE_DOCS_DIR = '../../docs/'
# file_name = dosc

In [4]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

# With a criteria (skip hidden files)
def is_not_hidden(path):
    return  not ( 'Icon' in path.name or '.DS_Store' in path.name or 'stylesheets'  in path.name or \
        'CNAME' in path.name or 'assets' in path.name or '.svg' in path.name or '.pages' in path.name)
    
# paths = DisplayablePath.make_tree(
#     Path(base_docs_dir),
#     criteria=is_not_hidden
# )
# for path in paths:
#     print(path.displayable())



# paths = DisplayablePath.make_tree(Path(base_docs_dir), criteria=is_not_hidden)
# for path in paths:
#     print(path.displayable())

def get_tree_structure(path_base=BASE_DOCS_DIR):
    
    paths = DisplayablePath.make_tree(Path(path_base), criteria=is_not_hidden)
    path_str = [p.displayable() for p in paths]
    # for path in paths:
    #     print(path.displayable())
    # return ''.join([p for p in path.displayable()])
    return '\n'.join(path_str)
    
tree_structure = get_tree_structure()
print(tree_structure)

docs/
├── index.md
├── javascripts/
│   ├── copy-link.js
│   └── mathjax.js
├── Managenai/
│   ├── brainstorming.md
│   ├── build_plan.md
│   ├── code_of_conduct.md
│   ├── contributing.md
│   ├── explorations_blog.md
│   ├── index.md
│   ├── project_requirements.md
│   └── site_graph.md
├── shared/
├── Understand/
│   ├── agents/
│   │   ├── actions_and_tools.md
│   │   ├── cognitive_architecture.md
│   │   ├── commercial.md
│   │   ├── environments.md
│   │   ├── evaluating_and_comparing.md
│   │   ├── examples.md
│   │   ├── image.png
│   │   ├── index.md
│   │   ├── memory.md
│   │   ├── rag.md
│   │   └── systems.md
│   ├── architectures/
│   │   ├── evaluating_and_comparing.md
│   │   ├── generation.md
│   │   ├── index.md
│   │   ├── llm_systems.md
│   │   ├── models/
│   │   │   ├── components.md
│   │   │   ├── developing_architectures.md
│   │   │   ├── diffusers.md
│   │   │   ├── gans.md
│   │   │   ├── gpt.md
│   │   │   ├── hybrid_models.md
│   │   │   ├── index.md
│   │ 

In [5]:
def get_file_name(file_path, base_dir=BASE_DOCS_DIR):
    # iterator for getting filenames
    return os.path.join(base_dir, file_path)

def get_structure_pattern(file_class=None):
    if file_class is None:
        file_class = 'index.md'
    patterns={}
    patterns['index.md'] = \
    """
NOTE: '-' is used to denote a general topic, sentence, or consideration but not considered a 'list' item.
## Executive Summary (TL;DR)
- Concise summary highlighting the essence of the topic and its significance.
- Designed for readability by a non-technical or executive-level audience.
- Utilize emojis, images, and visual elements effectively to emphasize key points.
- Include Mermaid diagrams where appropriate, or describe necessary images as `IMAGE: <image description>`.

## Practical Application and Usage
- Focus on providing immediately actionable guidance and high-priority examples.
- Extract and condense key usage instructions from earlier content into concise, actionable steps.
- Offer 'How-to' guides, quick-start tips, and links for direct application.

## Introduction and Relevance
- Thorough introduction to the topic, highlighting its relevance and importance.
- Discuss core components and their interplay within the broader context of Generative AI.

## Core Content and Results
- Detailed exploration of specific aspects under clear subheadings.
- Provide illustrations or diagrams (Mermaid or `IMAGE:<image description>`) for complex concepts.
- For extensive topics, include brief summaries and links to dedicated markdown files. If markdown files are already created, link to them here. If markdown files are needed, suggest them. 

## Technological Aspects
- Explore relevant tools, technologies, and methodologies.
- Highlight current trends and future directions in technology related to the topic.

## Background or Theoretical Foundation (if necessary)
- Delve into historical context and foundational theories.
- Clarify essential theoretical concepts and terminologies for comprehensive understanding.

## Ethical Considerations and Challenges
- Address ethical dilemmas, challenges, and potential risks.
- Discuss strategies for ethical practice and risk mitigation.

## Extended Examples (if applicable)
- Link to practical examples, simulations, or code snippets for hands-on understanding.
- Direct readers to external resources, tools, or demonstrations for further exploration.

## Advanced Topics and Further Exploration (if applicable)
- Present open challenges and future research directions.
- Deep dive into complex aspects with links to advanced readings and resources.

## FAQs and Common Queries
- Tackle frequently asked questions and common queries related to the section.

## Summary and Key Takeaways
- Recap the main points and emphasize the key messages from the section.

## References and Additional Reading
- List citations and provide links to source materials and further reading.

----


    """
    # patterns['glossary.md'] = \
    # """
    # """
    return patterns[file_class]


def get_markdown_text(markdown_file):
    with open(markdown_file, 'r') as f:
        markdown_text = f.read()
    # print(markdown_text)
    return markdown_text
# Could potentially do this is in few-shot prompt templates
# These should be generated

In [6]:
# role = "expert AI technology creator and communicator"

# project_name = "Managing Generative AI"
# project_goals = "Overall: Create an automated repository that is able to explain in plain-English and in code, "\
#                 "Generative AI and how to improve upon it. "
# present_task_description="Improve the markdown based on best understandings.\n"\
#                          "Be as honest and as accurate as possible. Be succinct in your responses. "

# from langchain import PromptTemplate
# # Idea
# # Select between prompt patterns
# # Chain select them more effectively. 
# # The present tree-structure:\n {tree_structure}\n 
# # Please use a heading/subheading structure that follows the general pattern : {structure_pattern}\n
# template = \
# """ 
# You are a {role}
# You are working on a project called: {project_name}\n
# You are part of a team working to: {project_goals}\n
# You are helping to: {present_task_description}\n
# You are helping to rewrite and expand a file called {file_name} 
# Here are some things we'd like you to be sure to do:
# * Please present ALL html links without changing the link's text. 
# * Preserve any urls or relative links without changing them. 
# * Be sure to use `##` `###` subheadings and appropriately to reference sections and subsections.
# * Please be sure to keep any amonitions like `!!!` and `???`.
# * Please reformat any bulleted lists of links where github links have `!!! abstract`, arxiv's have `!!! tip` and others have `!!! information`. 

# After the markdown When the text is presented (after >>>), please improve upon it. 
# If text is sparse or missing create a reasonable outline following the pattern above and fill it in.

# Markdown Input:\n
# >>>\n
# {markdown_text}"""

# prompt_template = PromptTemplate(
#     # input_variables=["role", "project_name", "project_goals", "present_task_description", "file_name", "tree_structure", "structure_pattern", "markdown_text", ],
#         input_variables=["role", "project_name", "project_goals", "present_task_description", "file_name",  "markdown_text", ],
#     template=template
# )
# file_from_base_dir = 'Use/deploying/index.md'
# file_from_base_dir = 'Use/redteaming.md'
# file_from_base_dir = 'Understand/agents/rag.md'
# file_name=get_file_name(file_from_base_dir)
# tree_structure=get_tree_structure()
# markdown_text=get_markdown_text(file_name)
# structure_pattern = get_structure_pattern()
# prompt=prompt_template.format(role=role,
#                               project_name=project_name,
#                        project_goals=project_goals,
#                        present_task_description=present_task_description,
#                        file_name=file_name,
#                        # tree_structure=tree_structure,
#                         structure_pattern=structure_pattern,
#                        markdown_text=markdown_text,)
# # The above is very verboase especially as it requires a lot of repeated typing of the same variables.
# # It also needs to work for variables that are only specified in the template. If they are not specified in the template, then they should be ignored.
# # There will a list of template lines that are appended to create the final prompt.
# # The template lines will be specified as a list of dictionaries.
# #  
# # Let's write this as a class




# #  write above but realizing template


In [7]:
class SmartPromptTemplate:
    def __init__(self, template_required, template_optional_dict, template_variable_independent):
        self.template_required = template_required if template_required is not None else ""
        self.template_optional_dict = template_optional_dict if template_optional_dict is not None else {}
        self.template_variable_independent = template_variable_independent if template_variable_independent is not None else ""
    
    def get_prompt(self, **kwargs):
        template_list = []
        for k, v in kwargs.items():
            if k in self.template_optional_dict.keys():
                template_list.append(self.template_optional_dict[k])
        begin_indicator = "\n What would you write given the requests above? \n>>>\n"
        
        template =   '\n'.join(template_list) + self.template_required  + "\n<<< end input \n" + \
                        self.template_variable_independent + begin_indicator
        prompt = template.format(**kwargs)
        return prompt

template_optional_dict = {

    'role': "You are a {role}",
    # 'project_name': "You are working on a project called: {project_name}\n",
    # 'project_goals': "You are part of a team working to: {project_goals}\n",
    'present_task_description': "You are helping to: {present_task_description}\n",
    'file_name': "You are helping to rewrite and expand a file called {file_name}\n",
    'structure_pattern': "Please use a heading/subheading structure that follows the general pattern : {structure_pattern}\n",
    'tree_structure': "The present tree-structure:\n {tree_structure}\n ",
    'markdown_text': "Markdown input \n>>>\n{markdown_text}"
}


template_variable_independent = \
"""
Things to keep in mind:
* present ALL html links without changing the link's text.
* Preserve any urls or relative links without changing them. 
* Be sure to use `##` `###` subheadings and appropriately to reference sections and subsections.
* keep ALL images `<img ...></img>` that are referenced in any manner.  
* Keep all code blocks that are referenced in any manner.
* Please be sure to keep any admonitions like `!!!` and `???`.
* Be as honest and as accurate as possible. 
* Be succinct in your responses. 
* Keep the ORIGINAL VOICE of the author there, and avoid unecessary changes to headings and subheadings. 
* If text is sparse or missing create a reasonable outline and follow it. 
* If you see MANAGEN (<and execute requests in trailing parenthesis>) then please evolve and expand upon the text in that area. 
* If you see any MANAGEN requests to make a mermaid diagram, please do so using the information that was provided.
* PRESERVE ALL STRUCTURED ADMONITIONS and following (that start with e.g. `!!!` and `???`) and DO NOT CHANGE THEM INTO BULLETS. Those need to be preserved.
* PRESERVE ALL INFORMATION IN MAIN MARKDOWN TEXT
* COPY ALL INFORMATON THAT IS IN ADMONITIONS!
* We'll get $1000 if we do this right, so let's do our best!
"""
# Please, do follow these instructions closely for it if we don't get this right, we might lose our job. 
# * reformat any bulleted lists of links where github links have `!!! abstract`, arxiv's have `!!! tip` and others have `!!! information`. 
# * Please be sure to keep any amonitions like `!!!` and `???`.
template_required = \
"""
"""

spt = SmartPromptTemplate(template_required=template_required, 
template_optional_dict=template_optional_dict, 
template_variable_independent=template_variable_independent)


In [8]:
role = "expert AI technology creator and communicator"

project_name = "Managing Generative AI"
project_goals = "Create an automated repository that is able to explain Generative AI "\
        "and how to improve upon it in plain-English and how to enable it from idea to product, as well as new and interesting research. "\
                
present_task_description="Improve the markdown based on best understandings."
                         
# file_from_base_dir = 'Use/deploying/index.md'
# file_from_base_dir = 'Use/deploying/libraries_and_tools.md'
# file_from_base_dir = 'Understand/data/preparation/synthetic.md'
# file_from_base_dir = 'Understand/prompting/index.md'
# file_from_base_dir = 'Understand/architectures/optimization/index.md'
# file_from_base_dir = 'Understand/architectures/rlhf.md'
# file_from_base_dir = 'Understand/agents/rag.md'
file_from_base_dir = 'Understand/Overview/challenges.md'
# file_from_base_dir = 'Understand/architectures/feedback.md'
file_name=get_file_name(file_from_base_dir)
tree_structure=get_tree_structure()
markdown_text=get_markdown_text(file_name)
structure_pattern = get_structure_pattern()
prompt = spt.get_prompt(role=role, 
    project_name=project_name,
    project_goals=project_goals,
    present_task_description=present_task_description,
    file_name=file_name,
#     tree_structure=tree_structure,
    markdown_text=markdown_text)

In [9]:
print(prompt)

You are a expert AI technology creator and communicator
You are helping to: Improve the markdown based on best understandings.

You are helping to rewrite and expand a file called ../../docs/Understanding/Overview/challenges.md

Markdown input 
>>>
There are a host of challenges associated with the use of GenAI at multiple levels. These challenges may also be appropriately considered _risks_ to add weight to their importance.  While references herein may provide partial solutions to these challenges, we do not suggest they are 'solved'. 

At the base level [technical challenges](#technical-challenges) relate to enabling the use of GenAI.
At a higher level [ethichal challenges](#ethical-challenges) relate to the risks that must be considered against benefits of using GenAI. 

## Technical Challenges

???+ important "Technical challenges with GenAI" technical-challenges-with-genai
    * Reducing [hallucinations](#hallucinations) and improving accuracy
    * Make LLMs generate results mor

In [10]:
markdown_text = openai(prompt)

In [11]:
# print(markdown_text)

In [12]:
#write the file to disk with a _temp suffix and then open it with a system call to tkdiff to visualize the two
# files side by side.
import os
import subprocess
import tempfile
import webbrowser

def write_to_file(file_name, text):
    with open(file_name, 'w') as f:
        f.write(text)
    return file_name

# Please be sure to run `homebrew install tkdiff` or otherwise install tkdiff on your computer
def open_with_tkdiff(file_name1, file_name2):
    subprocess.run(['tkdiff', file_name1, file_name2])

def make_name(file_name):
    base, ext = os.path.splitext(file_name)
    temp_name = base + '_temp0' + ext
    #check to see if it exists and if so, make a new name with a _temp# where # is the next available number
    count=0
    while os.path.exists(temp_name):
        count += 1
        
        temp_name = base + f'_temp{count}' + ext
    return temp_name
temp_name = make_name(file_name)
write_to_file(temp_name, markdown_text)


'../../docs/Understanding/Overview/challenges_temp0.md'

In [13]:

open_with_tkdiff(file_name, temp_name)


KeyboardInterrupt: 

In [ ]:
input_answer = input("Is the output correct Yes/no/deletefile? (y/n/d)")
## if the answer is y then move the temp-name to the original file name and delete the temp file
if input_answer == 'y':
    os.rename(temp_name, file_name)
    print(f"File {file_name} has been updated.")
else:
    print(f"File {file_name} has not been updated.")
    # if the answer is n then delete the temp file and do nothing
    if input_answer == 'd':
        os.remove(temp_name)
        print(f"File {temp_name} has been deleted.")


File ../../docs/Using/index.md has been updated.


BOT THAT LOOKS AT DIFFERENCES CHHUNK BY CHUNK AND AMENDS THEM. 
CREATE DIFF, ITERATE ON DIFF AND UPDATE MODIFIED DOCUMENT